<a href="https://colab.research.google.com/github/lonen8188/PythonAI/blob/5-3.Ensemble/hg_05_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 앙상블 : 단어 그대로 여러 단순한 모델을 결합하여 정확한 모델을 만드는 방법이다.
# 정형데이터 : 지금까지 학습한 수치자료가 있는 값
# 비정형데이터 : 데이터베이스나 엑셀로 표현하기 어려운 데이터(텍스트데이터, 디카사진, mp3 등.) -> 신경망 알고리즘
# 랜덤 포레스트 : 결정 트리를 랜덤하게 만들어 결정트리(나무)숲 을 만듬 -> 최종 예측
# 1000개의 샘플이 들어 있는 가방에서 100개을 샘플을 뽑을때 1개를 뽑고, 뽑앗던 1개를 다시 가방에 넣음
# 중복된 샘플을 뽑을 수 있음 -> 부트스트랩 샘플이라고 함
# 부트스트랩 : 데이터 세트에서 중복을 허용하여 데이터를 샘플링

# 분류 모델인 : RandomForestClassifier는 기본적으로 전체 특성 개수의 제곱근만큼 특성을 선택
# 즉 4개의 특성이 있다면 노드마드 2개를 랜덤하게 선택하여 사용
# 다만 회귀 모델인 RandomForestRegressor는 전체 특성을 사용

# 사이킷 런의 랜덤 포레스트는 기본적으로 100개의 결정 트리를 이런 방식으로 훈련한다.
# 그 다음 분류일 때는 각 트리의 클래스별 확률을 평균하여 가장 높은 확율을 가진 클래스를 예측으로 삼는다.
# 회귀일 때는 단순히 각 트리의 예측을 평균함.
# 분류 : 샘플을 몇개의 클래스 중 하나로 분류하는 문제
# 회귀 : 임의의 어떤 숫자를 예측하는 문제

# 와인찾기 데이터
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

# 훈련세트와 테스트세트로 나눔
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [2]:
from sklearn.model_selection import cross_validate # 크로스 검증용
from sklearn.ensemble import RandomForestClassifier # 100개의 결정 트리 사용

rf = RandomForestClassifier(n_jobs=-1, random_state=42) # n_jobs=-1 모든 cpu 사용
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
# return_train_score=True 검증 점수와 훈련 세트에 대한 점수도 리턴


print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# train_score 과대 적합 0.9973541965122431

0.9973541965122431 0.8905151032797809


In [3]:
# https://gsbang.tistory.com/entry/%EA%B8%B0%EA%B3%84%ED%95%99%EC%8A%B5-%EC%95%99%EC%83%81%EB%B8%94-%ED%95%99%EC%8A%B5-%EB%9E%9C%EB%8D%A4%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8Random-Forest#google_vignette

# n_estimators: 랜덤 포레스트에서 사용할 의사결정나무의 개수입니다. 이 값이 크면 클수록 모델의 성능은 좋아질 수 있지만, 계산 시간이 늘어납니다. (기본값: 100)
# criterion: 분할 품질을 측정하는 기능입니다. “gini”는 지니계수를 사용하고, “entropy”는 엔트로피를 사용합니다. (기본값: “gini”)
# max_depth: 나무의 최대 깊이를 제어합니다. 이 값이 너무 크면 과적합(overfitting)의 위험이 있고, 너무 작으면 성능이 떨어질 수 있습니다. None으로 설정하면, 모든 리프 노드가 순수해질 때까지 노드가 확장됩니다. (기본값: None)
# min_samples_split: 노드를 분할하는데 필요한 최소 샘플 수입니다. 이 값이 크면 나무의 깊이가 줄어들어 과적합을 방지할 수 있습니다. (기본값: 2)
# min_samples_leaf: 리프 노드에 있어야 하는 최소 샘플 수입니다. 이 값도 과적합을 제어하는데 사용됩니다. (기본값: 1)
# max_features: 각 노드에서 분할에 사용할 특성의 최대 수입니다. 이 값을 적절히 설정하여, 나무들 간의 상관관계를 줄일 수 있습니다. (기본값: "auto", 즉 sqrt(n_features))
# bootstrap: 부트스트랩 샘플링을 사용할지 여부를 결정합니다. True로 설정하면, 부트스트랩 샘플링을 사용하며, False로 설정하면 전체 데이터셋을 사용합니다. (기본값: True)
# oob_score: out-of-bag 샘플을 사용하여 일반화 정확도를 추정할지 여부를 결정합니다. (기본값: False)
# (out-of-bag : 학습에 사용되지 않은 데이터)
# n_jobs: 적합성과 예측을 위해 동시에 실행할 작업 수입니다. -1로 설정하면 모든 프로세서를 사용합니다. (기본값: None)
# random_state: 랜덤성을 제어하는 데 사용되는 시드 값입니다. 이 값을 고정하여 실험의 재현성을 보장할 수 있습니다. (기본값: None)
# verbose: 학습 과정에서 출력되는 텍스트 메시지의 상세 수준을 조절합니다. (기본값: 0)
# class_weight: 클래스 가중치를 설정할 수 있습니다. 불균형한 데이터셋을 처리할 때 유용합니다. (기본값: None)

rf.fit(train_input, train_target) # 훈련 후 특성 중요도 출력
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [4]:
# [0.12345626 0.86862934 0.0079144 ] LogisticRegression 이전 결과
# 알콜도수   , 당도      , pH 비교 해보자.
# 당도의 중요도가 감소하고, 알콜 도수와 pH 중요도가 상승함

# 특성 일부를 랜덤하게 선택하여 결정 트리를 훈련 하기 때문임
# RandomForestClassifier기능중에 자체적으로 모델을 평가하는 점수를 얻을 수 있다.
# OOB(Out Of Bag)부트스트랩에 포함되지 않고 남은 샘플 -> 결정 트리 평가용(검증 세트로 활용)
# oob_score=True
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


In [5]:
# 엑스트라 트리 : 100개의 결정 트리 훈련 -> 부트스트랩 샘플을 사용하지 않음(전체 훈련세트 사용)
# 대신 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할함!
# 엑스트라 트리의 결정 트리 splitter='random'

from sklearn.ensemble import ExtraTreesClassifier # ExtraTreesClassifier 엑스트라 트리

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# RandomForestClassifier와 결과가 비슷함. 대신 속도가 빠름

0.9974503966084433 0.8887848893166506


In [6]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [ ]:
# [0.12345626 0.86862934 0.0079144 ] LogisticRegression 이전 결과

In [7]:
# 그레이디언트 부스팅 Gradient Boosting(기울기)
# 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블
# 사이킷 런의 GradientBoostingClassifier는 기본적으로 깊이가 3인 결정 트리를 100개 사용
# 때문에 과대적합에 강하고 일반적인 높은 일반화 성능을 기대함

In [12]:
# Gradient는 경사 하강법을 사용하여 트리를 앙상블에 추가함.
# 분류 : 로지스틱 손실 함수, 회귀에서는 평균 제급 오차 함수를 사용
# 경사 하강법 손실 함수를 산으로 정의하고 가장 낮은 곳으로 찾아 내려오는 과정

from sklearn.ensemble import GradientBoostingClassifier # GradientBoostingClassifier 그레디언트 부스팅

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 과대 적합 해결

0.8881086892152563 0.8720430147331015


In [13]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # 교차 검증 점수 확인

0.9464595437171814 0.8780082549788999


In [14]:
gb.fit(train_input, train_target) # 훈련시작
print(gb.feature_importances_) # 당도에 의존도가 낮음

[0.15887763 0.6799705  0.16115187]


In [16]:
# 히스토그램 기반 그레이언트 부스팅 Histogram Gradient
# https://goatlab.tistory.com/entry/Machine-Learning-Histogram-Based-Gradient-Boosting-Ensembles

# 입력 특성을 256 구간으로 나눔 -> 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다.
# 특히 256구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용함
# HistGradientBoostingClassifier는 기본 매개변수에서 안정적인 성능을 얻을 수있다.
# HistGradientBoostingClassifier에는 트리의 개수를 지정하는데 n_estimators 대신 max_iter를 사용함(성능 항샹용)

# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # 과대 적합 억제 성공!

0.9321723946453317 0.8801241948619236


In [17]:
from sklearn.inspection import permutation_importance
# 특성 중요도 확인
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, #n_repeats=10랜덤하게 섞을수 있는 회수 기본5
                                random_state=42, n_jobs=-1)
print(result.importances_mean)
# 중요도,   평균,      표준편차

[0.08876275 0.23438522 0.08027708]


In [18]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)
# 테스트 세트에서 특성 중요도
# 중요도,   평균,      표준편차

[0.05969231 0.20238462 0.049     ]


In [19]:
hgb.score(test_input, test_target)

0.8723076923076923

In [20]:
# HistGradientBoostingGegressor 히스토그램 기반 그레이디언트 부스팅의 회귀 버전
# 사이킷 말고도 그레이디언트 부스팅 알고리즘을 구현한 라이브러리가 다수 존재
# XGBoost 대표적임
from xgboost import XGBClassifier
# tree_method='hist' 히스토그램 기반 그레이디언트 부스팅용
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


In [21]:
# LGBMClassifier ms에서 만든 LightGBM^2
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


0.935828414851749 0.8801251203079884
